In [ ]:
!pip install hsclient
!pip install numpy
!pip install scipy
!pip install requests

In [238]:
def create_timer_log(target, function_name, scheme="https://"):
    import requests
    import uuid

    uuid = uuid.uuid4()
    url = f"{scheme}{target}/hsapi/timer/{function_name}/{uuid}"
    r = requests.get(url)
    if not r.ok:
        logger.info("FAILED creating time log for url:")
        logger.info(url)
    else:
        logger.info(r.json()["message"])

# create_timer_log("localhost:8000", "test_naem", scheme="http://")

In [239]:
import logging
logger = logging.getLogger('stats-command')
logger.setLevel(logging.INFO)
handler = logging.FileHandler(filename="log/timer.log")
formatter = logging.Formatter("%(asctime)s - "
                              "%(levelname)s - "
                              "%(funcName)s - "
                              "line %(lineno)s - "
                              "%(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)

In [240]:
import pickle
targets = ["beta.hydroshare.org"]
meta_run_number = 0 # manually change for first run/ second
if meta_run_number == 0:
    meta_runs = [{}, {}] # pre and post
else:
    dbfile = open('pickle', 'rb')    
    meta_runs = pickle.load(dbfile)
    dbfile.close()
    
clients = {}
filename = "local-build-discover.sh"
diffs= {}

In [241]:
from hsclient import HydroShare
for target in targets:
    hs = HydroShare(host=target, port=443, protocol='https')
    hs.sign_in()
    hs.resources = []
    diffs[target] = []
    clients[target] = hs

In [242]:
from time import time
from functools import wraps
import numpy as np
import scipy
from copy import deepcopy

def timer_func(func): 
    @wraps(func)
    def wrap_func(*args, **kwargs):
        target = kwargs['target']
        create_timer_log(target, func.__name__)
        t1 = time() 
        result = func(*args, **kwargs)
        t2 = time()
        diff = t2-t1
        diffs[target].append(diff)
        message = f'Function {func.__name__!r} executed in {(diff):.4f}s'
        logger.info(message)
        print(message) 
        return result 
    return wrap_func


def create_res(hs, time=True, target=None):

    @timer_func
    def create_res_time(hs=None, target=None):
        resource = hs.create()
        hs.resources.append(resource)
    
    if time:
        create_res_time(hs=hs, target=target)
    else:
        resource = hs.create()
        hs.resources.append(resource)

@timer_func
def delete_resources(hs=None, target=None):
    for res in hs.resources:
        res.delete()

@timer_func
def download_all_resources(hs=None, target=None):
    for res in hs.resources:
        try:
            res.download()
        except Exception as e:
            print(e)

def download_single_resource(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def download_single_resource_inner(hs=res, target=None):
        res.download()

    download_single_resource_inner(hs=res, target=target)

def delete_single_resource(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def delete_single_resource_inner(hs=res, target=None):
        res.delete()

    delete_single_resource_inner(hs=res, target=target)

def upload_file(hs, target=None):
    create_res(hs, False)
    res = hs.resources[-1]

    @timer_func
    def upload_file_inner(hs=res, target=None):
        res.file_upload(filename)

    upload_file_inner(hs=res, target=target)


def create_dir_upload_file(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]

    @timer_func
    def create_dir_upload_file_inner(hs=res, target=None):
        res.folder_create('New_Folder')
        res.file_upload(filename, destination_path='New_Folder')
    create_dir_upload_file_inner(hs=res, target=target)

def upload_and_movefile(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    
    @timer_func
    def upload_and_movefile_inner(hs=res, target=None):
        res.folder_create('New_Folder')
        res.file_upload(filename, destination_path='New_Folder')
        file = res.file(path=f"New_Folder/{filename}]")
        res.file_rename(file, filename)
    upload_and_movefile_inner(hs=res, target=target)

def download_single_file(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def download_single_file_inner(hs=res, target=target):
        file = res.file(path=filename)
        res.file_download(file)
    download_single_file_inner(hs=res, target=target)

def delete_single_file(hs=None, target=None):
    create_res(hs, False)
    res = hs.resources[-1]
    res.file_upload(filename)

    @timer_func
    def delete_single_file_inner(hs=res, target=None):
        file = res.file(path=filename)
        res.file_delete(file)
    delete_single_file_inner(hs=res, target=target)
        

def run_comparisons(functions_to_run, runs=1):
    for funct in functions_to_run:
        message = f"********* Start {funct.__name__!r} *********"
        print(message)
        logger.info(message)
        for i in range(runs):
            for target in targets:
                client = clients[target]
                message = f"Starting run {i} of {funct.__name__} on {target}..."
                print(message)
                logger.info(message)
                funct(hs=client, target=target)
                print(diffs[target])
                logger.info(diffs[target])
        print("\n\n")
        # Store the diffs for later
        meta_runs[meta_run_number][funct.__name__] = deepcopy(diffs)
        for target in targets:
            diffs[target].clear()

def clear_resources():
    for target in targets:
        client = clients[target]
        client.resources = []
        diffs[target].clear()
  

In [246]:
# meta_runs = [{}, {}]

In [ ]:
clear_resources()
run_comparisons([download_single_resource, create_dir_upload_file, delete_single_file ], runs=3)
run_comparisons([delete_resources], runs=1)

In [ ]:
clear_resources()
run_comparisons([upload_file], runs=10)

In [ ]:
clear_resources()
run_comparisons([create_res, download_all_resources, delete_single_resource], runs=5)

In [ ]:
import pickle
file = open('pickle', 'ab')
    
# source, destination
pickle.dump(meta_runs, file)                    
file.close()

In [ ]:
# TODO: these ones still seem not to work...
#clear_resources()
#run_comparisons([upload_and_movefile, download_single_file ], runs=2)

In [ ]:
def run_stats():
    for func in meta_runs[0].keys():
        for idx, meta_run in enumerate(meta_runs):
            print(f"Analyzing function {func} on meta run number {idx}")
            target_obj = meta_run[func]
            for target, runs in target_obj.items():
                print(f"Average time for {func} on {target}: {sum(runs)/len(runs)}")
                print(f"Devation for {func} on {target}: {np.std(runs)}")
            if len(target_obj.keys()) == 2:
                print(f"Comparing runs of {func} between targets {target_obj.keys()}")
                # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
                result = scipy.stats.ttest_ind(target_obj.values()[0], target_obj.values()[1])
                print(result)
        if len(meta_runs) == 2:
            print("Comparing between two meta runs")
            if func in meta_runs[0] and func in meta_runs[1]:
                for target in meta_runs[0][func].keys():
                    print(f"Comparing meta runs for target {target}")
                    result = scipy.stats.ttest_ind(meta_runs[0][func][target], meta_runs[1][func][target])
                    print(result)
            else:
                print(f"{func} values are missing from one of the meta runs")
        else:
            print("Meta run number not = 2 so can't compare between meta runs")
        print("\n\n")

In [ ]:
run_stats()